<a href="https://colab.research.google.com/github/SBMagar/CarPrice-Prediction/blob/main/CarPrice_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Car price prediction with Linear Regression **

I am going to make a model for Car price prediction using Linear Regression Model with all correlation values plus error values

Due to some issues I have used this colab notebook and dataset from my drive linked with this notebook. 

I couldn't find the way to scrap hamrobazaar.com site for dataset so instead of this I have used foreign site to scrap and make my own dataset.

In [ ]:
import numpy as np
import json
import requests

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [ ]:
data = pd.DataFrame(pd.read_csv("/content/drive/My Drive/Colab Notebooks/Automobile_data.csv"))

In [ ]:
data.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.4,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.4,8.0,115,5500,18,22,17450


In [ ]:
list(data.columns)

['symboling',
 'normalized-losses',
 'make',
 'fuel-type',
 'aspiration',
 'num-of-doors',
 'body-style',
 'drive-wheels',
 'engine-location',
 'wheel-base',
 'length',
 'width',
 'height',
 'curb-weight',
 'engine-type',
 'num-of-cylinders',
 'engine-size',
 'fuel-system',
 'bore',
 'stroke',
 'compression-ratio',
 'horsepower',
 'peak-rpm',
 'city-mpg',
 'highway-mpg',
 'price']

In [ ]:
data['horsepower'] = pd.to_numeric(data['horsepower'], errors='coerce')
data['price'] = pd.to_numeric(data['price'], errors='coerce')


In [ ]:
data.dropna(subset=['horsepower', 'price'], inplace=True)

Price of car correlated to Engine horsepower... and its strength...


In [ ]:
from scipy.stats import pearsonr

In [ ]:
pearsonr(data['horsepower'], data['price'])

(0.8105330821322061, 1.189127827694731e-47)

The correlation coefficient of 0.81 shows a strong positive 
correlation

The p-value (probabilty of our data being uncorrelated): 1.189e-47 shows a low probability

In [ ]:
from bokeh.io import output_notebook
from bokeh.plotting import figure, show, ColumnDataSource


In [ ]:
output_notebook()

In [ ]:
source = ColumnDataSource(data=dict(
    x=data['horsepower'],
    y=data['price'],
    make=data['make']
))

# add tooltips to show infos for each datapoint
tooltips = [
    ('make', '@make'),
    ('horsepower', '$x'),
    ('price', '$y{$0}')
]

# create figure
p = figure(plot_width=600, plot_height=400, tooltips=tooltips)
# add axis labels
p.xaxis.axis_label = 'Horsepower'
p.yaxis.axis_label = 'Price'

# show datapoints as circles
p.circle('x', 'y', source=source, size=8, color='blue', alpha=0.5)

# show figure
show(p)

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.25)

In [ ]:
from sklearn import linear_model
model = linear_model.LinearRegression()

# reshape first array to 2D for .fit() method
training_x = np.array(train['horsepower']).reshape(-1, 1)
training_y = np.array(train['price'])

# perform linear fit/regression
model.fit(training_x, training_y)

# turn coefficient array which contains only one number into a number
slope = np.asscalar(np.squeeze(model.coef_))
intercept = model.intercept_
print('slope: ', slope, 'intercept: ', intercept)

slope:  170.33691526683205 intercept:  -4312.059347451552


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
  if sys.path[0] == '':


In [ ]:
from bokeh.models import Slope

best_fit = Slope(gradient=slope, y_intercept=intercept, line_color='red', line_width=3)
p.add_layout(best_fit)
show(p)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# define a function to generate a prediction and then compare the desired metrics
def predict_metrics(lr, x, y):
    pred = lr.predict(x)
    mae = mean_absolute_error(y, pred)
    mse = mean_squared_error(y, pred)
    r2 = r2_score(y, pred)
    return mae, mse, r2

training_mae, training_mse, training_r2 = predict_metrics(model, training_x, training_y)

# calculate with test data for comparison
test_x = np.array(test['horsepower']).reshape(-1, 1)
test_y = np.array(test['price'])
test_mae, test_mse, test_r2 = predict_metrics(model, test_x, test_y)

print('training mean error: ', training_mae, 'training mse: ', training_mse, 'training r2: ', training_r2)
print('test mean error: ', test_mae, 'test mse: ', test_mse, 'test r2: ', test_r2)

training mean error:  3355.2270850814903 training mse:  21982917.504038986 training r2:  0.6519027156060839
test mean error:  3283.5968277238358 test mse:  21000181.79367529 test r2:  0.6711823149729438


Add some features :-)

In [ ]:
cols = ['horsepower', 'engine-size', 'peak-rpm', 'length', 'width', 'height']
#preprocess the data
for col in cols:
    data[col] = pd.to_numeric(data[col], errors='coerce')

data.dropna(subset=['price', 'horsepower'], inplace=True)

for col in cols:
    print(col, pearsonr(data[col], data['price']))

horsepower (0.8105330821322061, 1.189127827694731e-47)
engine-size (0.8738869517981516, 1.26506744790746e-63)
peak-rpm (-0.10164886620219896, 0.15311824317200226)
length (0.6939647745646869, 6.398310603050564e-30)
width (0.7538710519013425, 8.679834788814272e-38)
height (0.13499022754460988, 0.05730390719825646)


drop peak-rpm & height from further analysis as they are weakly correlated

In [ ]:
# split data into train & test set
model_cols = ['horsepower', 'engine-size', 'length', 'width']
multi_x = np.column_stack(tuple(data[col] for col in model_cols))
multi_train_x, multi_test_x, multi_train_y, multi_test_y = train_test_split(multi_x, data['price'], test_size=0.25)

In [ ]:
multi_model = linear_model.LinearRegression()
multi_model.fit(multi_train_x, multi_train_y)
multi_intercept = multi_model.intercept_
multi_coeffs = dict(zip(model_cols, multi_model.coef_))
print('intercept: ', multi_intercept)
print('coefficients: ', multi_coeffs)

intercept:  -50291.06037105317
coefficients:  {'horsepower': 68.09746847117923, 'engine-size': 86.0130585057263, 'length': 16.369965963645534, 'width': 649.8368153196968}


In [ ]:
# calculate error metrics
multi_train_mae, multi_train_mse, multi_train_r2 = predict_metrics(multi_model, multi_train_x, multi_train_y)
multi_test_mae, multi_test_mse, multi_test_r2 = predict_metrics(multi_model, multi_test_x, multi_test_y)

print('training mean error: ', multi_train_mae, 'training mse: ', multi_train_mse, 'training r2: ', multi_train_r2)
print('test mean error: ', multi_test_mae, 'test mse: ', multi_test_mse, 'test r2: ', multi_test_r2)

training mean error:  2476.491662951245 training mse:  12402524.004767772 training r2:  0.7986320778617969
test mean error:  2359.2877045902715 test mse:  9826388.490419438 test r2:  0.8513719867771004
